Курсовая работа


Алгоритм в лоб

In [5]:
"""
Модуль для управления расписанием водителей автобусов.

Константы:
    DEFAULT_TRIP_DURATION (int): Стандартная длительность поездки в минутах
    DEFAULT_REST_DURATION (int): Стандартная длительность отдыха в минутах
    PEAK_HOURS (list): Список кортежей с временными интервалами часов пик
    PEAK_HOURS_ADDITIONAL_TIME (int): Дополнительное время в минутах для поездок в час пик
    BRUTE_FORCE_STEP (int): Шаг для поиска оптимального расписания
"""


DEFAULT_TRIP_DURATION = 90
DEFAULT_REST_DURATION = 10
PEAK_HOURS = [("07:00", "09:00"), ("17:00", "19:00")]
PEAK_HOURS_ADDITIONAL_TIME = 5
BUS_COUNT = 2
DRIVERS_PER_GROUP_COUNT = 4
BRUTE_FORCE_STEP = 35

In [6]:
from datetime import datetime, timedelta
import pandas as pd


def is_peak_hour(current_time: datetime) -> bool:
    """
    Проверяет, является ли указанное время часом пик.

    Args:
        current_time (datetime): Время для проверки

    Returns:
        bool: True если время попадает в час пик, False в противном случае
    """

    for start, end in PEAK_HOURS:
        peak_start = datetime.strptime(start, "%H:%M").time()
        peak_end = datetime.strptime(end, "%H:%M").time()
        if peak_start <= current_time.time() < peak_end:
            return True
    return False


def get_trip_duration(
    start_time: datetime, base_duration: int = DEFAULT_TRIP_DURATION
) -> int:
    """
    Рассчитывает фактическую длительность поездки с учетом часов пик.

    Args:
        start_time (datetime): Время начала поездки
        base_duration (int): Базовая длительность поездки в минутах

    Returns:
        int: Фактическая длительность поездки в минутах
    """
    end_time = start_time + timedelta(minutes=base_duration)

    if is_peak_hour(start_time) or is_peak_hour(end_time):
        base_duration += 5

    return base_duration


class TimeInterval:
    """
    Класс для представления временного интервала.

    Attributes:
        start (datetime): Время начала интервала
        end (datetime): Время окончания интервала
    """

    def __init__(self, start: datetime, end: datetime):
        """
        Args:
            start (datetime): Время начала интервала
            end (datetime): Время окончания интервала
        """
        self.start = start
        self.end = end

    def __repr__(self):
        return f"{self.start.strftime('%H:%M')} - {self.end.strftime('%H:%M')}"

    def overlaps(self, other) -> bool:
        """
        Проверяет, пересекается ли текущий интервал с другим.

        Args:
            other (TimeInterval): Другой временной интервал

        Returns:
            bool: True если интервалы пересекаются, False в противном случае
        """
        return self.start < other.end and self.end > other.start


class Bus:
    """
    Класс для представления автобуса.

    Attributes:
        id (int): Идентификатор автобуса
        capacity (int): Вместимость автобуса
        speed (int): Скорость автобуса
        type (str): Тип автобуса
    """

    def __init__(
        self, id: int, capacity: int = 100, speed: int = 60, type: str = "Электробус"
    ):
        """
        Args:
            id (int): Идентификатор автобуса
            capacity (int, optional): Вместимость. По умолчанию 100
            speed (int, optional): Скорость. По умолчанию 60
            type (str, optional): Тип автобуса. По умолчанию "Электробус"
        """
        self.id = id
        self.capacity = capacity
        self.speed = speed
        self.type = type


class Driver:
    """
    Класс для представления водителя.

    Attributes:
        id (int): Идентификатор водителя
        bus_id (int): Идентификатор закрепленного автобуса
        work_time (list): Список рабочих интервалов
        rest_time (list): Список интервалов отдыха
        type (str): Тип смены ("12_hr" или "8_hr")
        experience (int): Опыт работы
    """

    def __init__(
        self,
        id: int,
        bus_id: int,
        work_time: [(TimeInterval, int)] = None,
        rest_time: [TimeInterval] = None,
        type: str = "12_hr",
        expirience: int = 0,
    ):
        self.id = id
        self.bus_id = bus_id
        self.work_time = work_time if work_time is not None else []
        self.rest_time = rest_time if rest_time is not None else []
        self.type = type
        self.expirience = expirience

    def get_trip_intevals(self) -> [TimeInterval]:
        """
        Возвращает список всех рабочих интервалов водителя.

        Returns:
            list: Список объектов TimeInterval
        """
        return [interval for interval, bus_id in self.work_time]

    def get_last_trip_inteval(self) -> TimeInterval | None:
        """
        Возвращает последний рабочий интервал водителя.

        Returns:
            TimeInterval: Последний интервал или None, если интервалов нет
        """
        return self.get_trip_intevals()[-1] if self.get_trip_intevals() else None

    def get_work_hours(self) -> int | None:
        """
        Возвращает количество рабочих часов в зависимости от типа смены.

        Returns:
            int: Количество часов или None для неизвестного типа смены
        """
        if self.type == "12_hr":
            return 12
        elif self.type == "8_hr":
            return 8

        return None

    def get_rest_minutes(self) -> int | None:
        """
        Возвращает длительность перерыва в зависимости от типа смены.

        Returns:
            int: Количество минут или None для неизвестного типа смены
        """
        if self.type == "12_hr":
            return 10
        elif self.type == "8_hr":
            return 60
        return None

    def generate_schedule(self, start_time: str) -> None:
        """
        Генерирует расписание работы и отдыха водителя.

        Args:
            start_time (str): Время начала работы в формате "HH:MM"

        Note:
            Метод создает расписание с учетом:
            - Типа смены (8 или 12 часов)
            - Обязательных перерывов
            - Часов пик
            - Максимальной продолжительности работы
        """

        self.work_time.clear()
        self.rest_time.clear()

        start = datetime.strptime(start_time, "%H:%M")
        end_time = start + timedelta(hours=self.get_work_hours())

        current_time = start
        trip_count = 0

        while current_time < end_time:
            trip_duration = get_trip_duration(current_time)
            trip_start = current_time
            trip_end = current_time + timedelta(minutes=trip_duration)

            if trip_end.time() < trip_start.time():
                self.work_time.append((TimeInterval(trip_start, trip_end), self.bus_id))
                break

            self.work_time.append((TimeInterval(trip_start, trip_end), self.bus_id))

            trip_count += 1
            current_time = trip_end

            if self.type == "12_hr" and trip_count % 2 == 0:
                rest_start = current_time
                rest_end = current_time + timedelta(minutes=self.get_rest_minutes())

                if rest_end.time() < rest_start.time():
                    rest_end = datetime.strptime("00:00", "%H:%M").time()

                    self.rest_time.append(TimeInterval(rest_start, rest_end))
                    break

                self.rest_time.append(TimeInterval(rest_start, rest_end))
                current_time = rest_end


In [7]:
def check_continuous_coverage(intervals) -> tuple:
    """
    Проверяет непрерывность покрытия рабочих смен в течение суток.

    Функция сортирует интервалы по времени начала и проверяет наличие
    промежутков между последовательными интервалами. Также проверяется,
    что последний интервал достигает конца суток.

    Args:
        intervals (list): Список объектов TimeInterval, представляющих рабочие смены

    Returns:
        tuple: Кортеж из трех элементов:
            - bool: True если покрытие непрерывно, False если есть разрывы
            - str|None: Время окончания интервала, где найден разрыв (формат "HH:MM")
            - str|None: Время начала следующего интервала (формат "HH:MM")
    """
    intervals.sort(key=lambda x: x.start)

    for i in range(len(intervals) - 1):
        if (
            intervals[i].end < intervals[i + 1].start
            and intervals[i].start < intervals[i].end
        ):
            return (
                False,
                intervals[i].end.strftime("%H:%M"),
                intervals[i + 1].start.strftime("%H:%M"),
            )

    if (
        intervals[-1].end.time() < datetime.strptime("23:59", "%H:%M").time()
        and intervals[-1].start.time() < intervals[-1].end.time()
    ):
        return False, intervals[-1].end.strftime("%H:%M"), "23:59"

    return True, None, None


def get_all_trips_intervals(drivers) -> [TimeInterval]:
    """
    Собирает все рабочие интервалы всех водителей в единый список.

    Args:
        drivers (list): Список объектов Driver

    Returns:
        list: Список всех рабочих интервалов (объекты TimeInterval)
    """
    intervals = []

    for driver in drivers:
        intervals.extend(driver.get_trip_intevals())

    return intervals


def find_valid_schedule(drivers, drivers_start_time) -> bool:
    """
    Ищет валидное расписание для группы водителей методом перебора времени начала смен.

    Функция перебирает различные комбинации времени начала смен для водителей,
    пытаясь найти такое расписание, которое обеспечивает непрерывное покрытие
    рабочего дня и удовлетворяет дополнительным условиям.

    Args:
        drivers (list): Список объектов Driver
        drivers_start_time (dict): Словарь с начальным временем смен для каждого водителя
                                 {id_водителя: "HH:MM"}

    Returns:
        bool: True если найдено валидное расписание, False если нет

    Note:
        - Перебор времени осуществляется с шагом BRUTE_FORCE_STEP минут
        - Проверяются следующие условия:
            1. Непрерывность покрытия всех смен
            2. Время окончания смены водителя №3 должно быть раньше 23:45
            3. Все смены должны заканчиваться в тот же день, когда начались
    """
    start_time = datetime.strptime("00:00", "%H:%M")
    end_time = datetime.strptime("23:55", "%H:%M")
    time_step = timedelta(minutes=BRUTE_FORCE_STEP)

    all_times = []
    current_time = start_time
    while current_time <= end_time:
        all_times.append(current_time.strftime("%H:%M"))
        current_time += time_step

    for time_2 in all_times:
        drivers_start_time[2] = time_2
        for time_3 in all_times:
            drivers_start_time[3] = time_3
            for time_4 in all_times:
                drivers_start_time[4] = time_4

                for driver in drivers:
                    driver.generate_schedule(drivers_start_time[driver.id])

                all_intervals = get_all_trips_intervals(drivers)
                is_continuous_coverage, start_time, end_time = (
                    check_continuous_coverage(all_intervals)
                )

                driver_3_last_trip = drivers[2].get_last_trip_inteval()
                if (
                    is_continuous_coverage
                    and driver_3_last_trip
                    and driver_3_last_trip.end.time()
                    < datetime.strptime("23:45", "%H:%M").time()
                    and driver_3_last_trip.start.time() < driver_3_last_trip.end.time()
                ):
                    return True

    return False


In [8]:
"""
Инициализация и тестирование системы расписания водителей.

Этот блок кода выполняет следующие действия:
1. Создает автобусы и водителей
2. Устанавливает начальное время работы
3. Ищет валидное расписание
4. Выводит результаты

Константы:
    BUS_COUNT (int): Количество автобусов (2)
    DRIVERS_PER_GROUP_COUNT (int): Количество водителей в группе (4)

Структура данных:
    buses (list): Список объектов Bus
        - Создается два автобуса с ID 1 и 2

    drivers (list): Список объектов Driver
        - Создается 4 водителя
        - Четные ID привязаны к автобусу №2
        - Нечетные ID привязаны к автобусу №1

    drivers_start_time (dict): Начальное время старта для каждого водителя
        - Ключи: ID водителей (1-4)
        - Значения: время начала смены в формате "HH:MM"

Вывод результатов:
    Успешный случай:
        - Для каждого водителя выводится:
            * Время поездок с номером автобуса
            * Время перерывов

    Неуспешный случай:
        - Сообщение о невозможности найти валидное расписание

Примечание:
    Валидное расписание должно обеспечивать:
    - Непрерывное покрытие рабочего дня
    - Корректное распределение автобусов
    - Соблюдение требований к времени работы и отдыха
"""
buses = [Bus(id=x) for x in range(1, BUS_COUNT + 1)]
drivers = [
    Driver(id=driver_id, bus_id=(2 if driver_id % 2 == 0 else 1))
    for driver_id in range(1, DRIVERS_PER_GROUP_COUNT + 1)
]


drivers_start_time = {1: "00:00", 2: "00:00", 3: "00:00", 4: "00:00"}


if find_valid_schedule(drivers, drivers_start_time):
    for d in drivers:
        for interval in d.get_trip_intevals():
            print(f"Водитель {d.id}, поездка: {interval}, автобус: №{d.bus_id}")
        for interval in d.rest_time:
            print(f"Водитель {d.id}, отдых: {interval}")
else:
    print("Не удалось найти валидное расписание")

Водитель 1, поездка: 00:00 - 01:30, автобус: №1
Водитель 1, поездка: 01:30 - 03:00, автобус: №1
Водитель 1, поездка: 03:10 - 04:40, автобус: №1
Водитель 1, поездка: 04:40 - 06:10, автобус: №1
Водитель 1, поездка: 06:20 - 07:55, автобус: №1
Водитель 1, поездка: 07:55 - 09:30, автобус: №1
Водитель 1, поездка: 09:40 - 11:10, автобус: №1
Водитель 1, поездка: 11:10 - 12:40, автобус: №1
Водитель 1, отдых: 03:00 - 03:10
Водитель 1, отдых: 06:10 - 06:20
Водитель 1, отдых: 09:30 - 09:40
Водитель 1, отдых: 12:40 - 12:50
Водитель 2, поездка: 00:35 - 02:05, автобус: №2
Водитель 2, поездка: 02:05 - 03:35, автобус: №2
Водитель 2, поездка: 03:45 - 05:15, автобус: №2
Водитель 2, поездка: 05:15 - 06:45, автобус: №2
Водитель 2, поездка: 06:55 - 08:30, автобус: №2
Водитель 2, поездка: 08:30 - 10:05, автобус: №2
Водитель 2, поездка: 10:15 - 11:45, автобус: №2
Водитель 2, поездка: 11:45 - 13:15, автобус: №2
Водитель 2, отдых: 03:35 - 03:45
Водитель 2, отдых: 06:45 - 06:55
Водитель 2, отдых: 10:05 - 10:15
В

In [9]:
"""
Создание и форматирование недельного расписания водителей в виде таблицы pandas DataFrame.

Основные этапы:
1. Инициализация водителей
2. Определение графика работы групп
3. Создание структуры таблицы
4. Заполнение данных
5. Форматирование и стилизация

Структура данных:
    all_drivers (list): Список из 12 водителей
        - 3 группы по 4 водителя
        - Четные ID привязаны к автобусу №2
        - Нечетные ID привязаны к автобусу №1

    work_schedule (dict): График работы для каждой группы
        - Группа 1: Понедельник, Четверг, Воскресенье
        - Группа 2: Вторник, Пятница
        - Группа 3: Среда, Суббота

    days (list): Список дней недели для колонок таблицы

    columns (MultiIndex): Двухуровневый индекс колонок
        - Первый уровень: дни недели
        - Второй уровень: "График работы" и "Время отдыха"

Формат таблицы:
    Индекс:
        - Информация о водителе (номер, автобус, группа)

    Колонки:
        - Основные: дни недели
        - Подколонки: график работы и время отдыха

    Содержимое ячеек:
        - Для рабочих дней: времена поездок и перерывов
        - Для выходных: "Выходной" и "---"

Стилизация:
    - Центрирование текста
    - Границы ячеек
    - Разные размеры шрифта для заголовков
    - Фоновый цвет для основных заголовков
    - Автоматический перенос текста

Примечание:
    Расписание генерируется на основе начального времени из drivers_start_time
    с учетом принадлежности водителя к определенной группе.
"""


all_drivers = []
for group in range(3):
    for driver_id in range(1, 5):
        current_id = driver_id + (group * 4)
        bus_id = 2 if driver_id % 2 == 0 else 1
        all_drivers.append(Driver(id=current_id, bus_id=bus_id))


work_schedule = {
    1: ["Понедельник", "Четверг", "Воскресенье"],
    2: ["Вторник", "Пятница"],
    3: ["Среда", "Суббота"],
}


days = [
    "Понедельник",
    "Вторник",
    "Среда",
    "Четверг",
    "Пятница",
    "Суббота",
    "Воскресенье",
]
columns = pd.MultiIndex.from_product([days, ["График работы", "Время отдыха"]])


data = []
for driver in all_drivers:
    group_num = (driver.id - 1) // 4 + 1

    driver.generate_schedule(
        drivers_start_time[driver.id % 4 if driver.id % 4 != 0 else 4]
    )

    row_data = {}

    driver_info = f"Водитель №{driver.id}\nАвтобус №{driver.bus_id}\nГруппа {group_num}"
    row_data["Информация"] = driver_info

    for day in days:
        if day in work_schedule[group_num]:
            work_times = [
                f"{interval.start.strftime('%H:%M')}-{interval.end.strftime('%H:%M')}"
                for interval, _ in driver.work_time
            ]
            rest_times = [
                f"{interval.start.strftime('%H:%M')}-{interval.end.strftime('%H:%M')}"
                for interval in driver.rest_time
            ]

            row_data[(day, "График работы")] = "\n".join(work_times)
            row_data[(day, "Время отдыха")] = "\n".join(rest_times)
        else:
            row_data[(day, "График работы")] = "Выходной"
            row_data[(day, "Время отдыха")] = "---"

    data.append(row_data)

df = pd.DataFrame(data)
df.set_index("Информация", inplace=True)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

styled_df = df.style.set_properties(
    **{"text-align": "center", "white-space": "pre-wrap"}
).set_table_styles(
    [
        {"selector": "th", "props": [("text-align", "center")]},
        {"selector": "", "props": [("border", "1px solid black")]},
        {"selector": "th.col_heading", "props": [("font-size", "11pt")]},
        {
            "selector": "th.col_heading.level0",
            "props": [("font-size", "12pt"), ("background-color", "#f0f0f0")],
        },
    ]
)

print("\nРасписание работы водителей на неделю:")
display(styled_df)


Расписание работы водителей на неделю:


,"('Понедельник', 'График работы')","('Понедельник', 'Время отдыха')","('Вторник', 'График работы')","('Вторник', 'Время отдыха')","('Среда', 'График работы')","('Среда', 'Время отдыха')","('Четверг', 'График работы')","('Четверг', 'Время отдыха')","('Пятница', 'График работы')","('Пятница', 'Время отдыха')","('Суббота', 'График работы')","('Суббота', 'Время отдыха')","('Воскресенье', 'График работы')","('Воскресенье', 'Время отдыха')"
Информация,,,,,,,,,,,,,,
Водитель №1 Автобус №1 Группа 1,00:00-01:30 01:30-03:00 03:10-04:40 04:40-06:10 06:20-07:55 07:55-09:30 09:40-11:10 11:10-12:40,03:00-03:10 06:10-06:20 09:30-09:40 12:40-12:50,Выходной,---,Выходной,---,00:00-01:30 01:30-03:00 03:10-04:40 04:40-06:10 06:20-07:55 07:55-09:30 09:40-11:10 11:10-12:40,03:00-03:10 06:10-06:20 09:30-09:40 12:40-12:50,Выходной,---,Выходной,---,00:00-01:30 01:30-03:00 03:10-04:40 04:40-06:10 06:20-07:55 07:55-09:30 09:40-11:10 11:10-12:40,03:00-03:10 06:10-06:20 09:30-09:40 12:40-12:50
Водитель №2 Автобус №2 Группа 1,00:35-02:05 02:05-03:35 03:45-05:15 05:15-06:45 06:55-08:30 08:30-10:05 10:15-11:45 11:45-13:15,03:35-03:45 06:45-06:55 10:05-10:15 13:15-13:25,Выходной,---,Выходной,---,00:35-02:05 02:05-03:35 03:45-05:15 05:15-06:45 06:55-08:30 08:30-10:05 10:15-11:45 11:45-13:15,03:35-03:45 06:45-06:55 10:05-10:15 13:15-13:25,Выходной,---,Выходной,---,00:35-02:05 02:05-03:35 03:45-05:15 05:15-06:45 06:55-08:30 08:30-10:05 10:15-11:45 11:45-13:15,03:35-03:45 06:45-06:55 10:05-10:15 13:15-13:25
Водитель №3 Автобус №1 Группа 1,08:45-10:20 10:20-11:50 12:00-13:30 13:30-15:00 15:10-16:40 16:40-18:15 18:25-20:00 20:00-21:30,11:50-12:00 15:00-15:10 18:15-18:25 21:30-21:40,Выходной,---,Выходной,---,08:45-10:20 10:20-11:50 12:00-13:30 13:30-15:00 15:10-16:40 16:40-18:15 18:25-20:00 20:00-21:30,11:50-12:00 15:00-15:10 18:15-18:25 21:30-21:40,Выходной,---,Выходной,---,08:45-10:20 10:20-11:50 12:00-13:30 13:30-15:00 15:10-16:40 16:40-18:15 18:25-20:00 20:00-21:30,11:50-12:00 15:00-15:10 18:15-18:25 21:30-21:40
Водитель №4 Автобус №2 Группа 1,11:40-13:10 13:10-14:40 14:50-16:20 16:20-17:55 18:05-19:40 19:40-21:10 21:20-22:50 22:50-00:20,14:40-14:50 17:55-18:05 21:10-21:20,Выходной,---,Выходной,---,11:40-13:10 13:10-14:40 14:50-16:20 16:20-17:55 18:05-19:40 19:40-21:10 21:20-22:50 22:50-00:20,14:40-14:50 17:55-18:05 21:10-21:20,Выходной,---,Выходной,---,11:40-13:10 13:10-14:40 14:50-16:20 16:20-17:55 18:05-19:40 19:40-21:10 21:20-22:50 22:50-00:20,14:40-14:50 17:55-18:05 21:10-21:20
Водитель №5 Автобус №1 Группа 2,Выходной,---,00:00-01:30 01:30-03:00 03:10-04:40 04:40-06:10 06:20-07:55 07:55-09:30 09:40-11:10 11:10-12:40,03:00-03:10 06:10-06:20 09:30-09:40 12:40-12:50,Выходной,---,Выходной,---,00:00-01:30 01:30-03:00 03:10-04:40 04:40-06:10 06:20-07:55 07:55-09:30 09:40-11:10 11:10-12:40,03:00-03:10 06:10-06:20 09:30-09:40 12:40-12:50,Выходной,---,Выходной,---
Водитель №6 Автобус №2 Группа 2,Выходной,---,00:35-02:05 02:05-03:35 03:45-05:15 05:15-06:45 06:55-08:30 08:30-10:05 10:15-11:45 11:45-13:15,03:35-03:45 06:45-06:55 10:05-10:15 13:15-13:25,Выходной,---,Выходной,---,00:35-02:05 02:05-03:35 03:45-05:15 05:15-06:45 06:55-08:30 08:30-10:05 10:15-11:45 11:45-13:15,03:35-03:45 06:45-06:55 10:05-10:15 13:15-13:25,Выходной,---,Выходной,---
Водитель №7 Автобус №1 Группа 2,Выходной,---,08:45-10:20 10:20-11:50 12:00-13:30 13:30-15:00 15:10-16:40 16:40-18:15 18:25-20:00 20:00-21:30,11:50-12:00 15:00-15:10 18:15-18:25 21:30-21:40,Выходной,---,Выходной,---,08:45-10:20 10:20-11:50 12:00-13:30 13:30-15:00 15:10-16:40 16:40-18:15 18:25-20:00 20:00-21:30,11:50-12:00 15:00-15:10 18:15-18:25 21:30-21:40,Выходной,---,Выходной,---
Водитель №8 Автобус №2 Группа 2,Выходной,---,11:40-13:10 13:10-14:40 14:50-16:20 16:20-17:55 18:05-19:40 19:40-21:10 21:20-22:50 22:50-00:20,14:40-14:50 17:55-18:05 21:10-21:20,Выходной,---,Выходной,---,11:40-13:10 13:10-14:40 14:50-16:20 16:20-17:55 18:05-19:40 19:40-21:10 21:20-22:50 22:50-00:20,14:40-14:50 17:55-18:05 21:10-21:20,Выходной,---,Выхо

In [10]:
from openpyxl.styles import Alignment, PatternFill, Font, Border, Side
from openpyxl.utils import get_column_letter

"""
Сохранение и форматирование таблицы расписания в Excel-файл.

Импорты:
    openpyxl.styles: Классы для стилизации Excel
    openpyxl.utils: Утилиты для работы с Excel

Основные этапы:
1. Создание Excel-файла
2. Запись данных из DataFrame
3. Применение стилей форматирования
4. Настройка размеров колонок

Форматирование:
    border: Тонкие границы для всех ячеек
    header_fill: Серый фон для заголовков
    alignment: Центрирование текста и автоперенос
    font: Размер шрифта 11pt для заголовков

Особенности:
    - Используется openpyxl для расширенного форматирования
    - Автоматический расчет ширины колонок
    - Двухуровневые заголовки (строки 1 и 2)
    - Сохранение в файл 'schedule.xlsx'

Примечание:
    Форматирование применяется после записи данных
    и сохраняет все стили из исходного DataFrame
"""

with pd.ExcelWriter("schedule.xlsx", engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name="Расписание")

    worksheet = writer.sheets["Расписание"]

    border = Border(
        left=Side(style="thin"),
        right=Side(style="thin"),
        top=Side(style="thin"),
        bottom=Side(style="thin"),
    )

    header_fill = PatternFill(
        start_color="F0F0F0", end_color="F0F0F0", fill_type="solid"
    )

    for row in worksheet.iter_rows():
        for cell in row:
            cell.alignment = Alignment(
                horizontal="center", vertical="center", wrap_text=True
            )

            cell.border = border

            if cell.row in [1, 2]:
                cell.font = Font(size=11)
                cell.fill = header_fill

    for column in worksheet.columns:
        max_length = 0
        column_letter = get_column_letter(column[0].column)

        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except Exception as e:
                print(e)

        adjusted_width = max_length + 2
        worksheet.column_dimensions[column_letter].width = adjusted_width

print("Расписание сохранено в файл 'schedule.xlsx'")

Расписание сохранено в файл 'schedule.xlsx'


Генетический алгоритм

In [1]:
import random
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Константы
NUM_BUSES = 8
NUM_DRIVERS = 12
NUM_8HR_DRIVERS = 6
NUM_12HR_DRIVERS = NUM_DRIVERS - NUM_8HR_DRIVERS
SHIFT_DURATIONS = {"8hr": 8, "12hr": 12}
DAYS_OF_WEEK = ["Понедельник", "Вторник", "Среда", "Четверг", "Пятница", "Суббота", "Воскресенье"]
TIME_SLOTS = range(24)

# Параметры генетического алгоритма
POPULATION_SIZE = 100
MAX_GENERATIONS = 1000
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.9
MAX_GENERATIONS_WITHOUT_IMPROVING = 1000
ELITE_SIZE = POPULATION_SIZE // 5

def is_rest_day(driver_id, day_idx):
    if driver_id < NUM_8HR_DRIVERS:  # 8-часовые водители (0-5)
        # Отдыхают в выходные
        return DAYS_OF_WEEK[day_idx] in ["Суббота", "Воскресенье"]
    
    elif driver_id < NUM_8HR_DRIVERS + 2:  # Первая группа 12-часовых (6-7)
        # Работают в понедельник, четверг, воскресенье
        return DAYS_OF_WEEK[day_idx] not in ["Понедельник", "Четверг", "Воскресенье"]
    
    elif driver_id < NUM_8HR_DRIVERS + 4:  # Вторая группа 12-часовых (8-9)
        # Работают во вторник, пятницу
        return DAYS_OF_WEEK[day_idx] not in ["Вторник", "Пятница"]
    
    else: 
        return DAYS_OF_WEEK[day_idx] not in ["Среда", "Суббота"]


def adaptive_mutation_rate(generations_without_improvement):
    """Увеличивает вероятность мутации при стагнации"""
    base_rate = 0.1
    return min(base_rate * (1 + generations_without_improvement / 10), 0.5)

def two_point_crossover(parent1, parent2):
    """Двухточечный кроссовер для лучшего смешивания генов"""
    point1 = random.randint(0, len(parent1) - 2)
    point2 = random.randint(point1 + 1, len(parent1) - 1)
    return parent1[:point1] + parent2[point1:point2] + parent1[point2:]

def check_shift_overlap(schedule):
    """Проверяет и штрафует пересечение смен водителей на одном автобусе"""
    penalty = 0
    for day_schedule in schedule:
        bus_shifts = {}
        for shift in day_schedule:
            if not shift["rest"]:
                bus = shift["bus"]
                if bus not in bus_shifts:
                    bus_shifts[bus] = []
                bus_shifts[bus].append((shift["start"], shift["end"], shift["driver"]))
        
        for bus_id, shifts in bus_shifts.items():
            shifts.sort()
            for i in range(len(shifts)-1):
                if shifts[i][1] > shifts[i+1][0]:
                    penalty += 30
    return penalty

def fitness(schedule):
    PENALTIES = {
        'bus_shortage': 10,
        'weekend_work': 15,
        'rest_violation': 20,
        'peak_hours_shortage': 25,
        'shift_overlap': 30,
        'no_workers_day': 10000 
    }
    
    BONUSES = {
        'weekend_rest': 150,
        'proper_rest_12hr': 140,
    }
    
    PEAK_HOURS = {
        'morning': range(7, 10),
        'evening': range(17, 20)
    }

    penalty = 0
    bonus = 0
    bus_coverage = {day: {hour: [] for hour in TIME_SLOTS} for day in DAYS_OF_WEEK}


    for day_idx, daily_schedule in enumerate(schedule):
        working_drivers = sum(1 for driver_schedule in daily_schedule if not driver_schedule["rest"])
        if working_drivers == 0:
            penalty += PENALTIES['no_workers_day']

    weekend_rest_violation = False
    for day_idx, day in enumerate(DAYS_OF_WEEK):
        if day in ["Суббота", "Воскресенье"]:
            for driver_schedule in schedule[day_idx]:
                if (driver_schedule["driver"] < NUM_8HR_DRIVERS and 
                    not driver_schedule["rest"]):
                    weekend_rest_violation = True
                    break
    
    if not weekend_rest_violation:
        bonus += BONUSES['weekend_rest']

    for driver_id in range(NUM_8HR_DRIVERS, NUM_DRIVERS):
        proper_rest = True
        for day_idx in range(1, len(DAYS_OF_WEEK)):
            if not schedule[day_idx][driver_id]["rest"]:
                for prev_day in range(max(0, day_idx - 2), day_idx):
                    if not schedule[prev_day][driver_id]["rest"]:
                        proper_rest = False
                        break
        if proper_rest:
            bonus += BONUSES['proper_rest_12hr']

    penalty += check_shift_overlap(schedule)

    for day_idx, daily_schedule in enumerate(schedule):
        for driver_schedule in daily_schedule:
            if driver_schedule["rest"]:
                continue

            bus = driver_schedule["bus"]
            start, end = driver_schedule["start"], driver_schedule["end"]

            if bus is not None:
                hours = range(start, end + 1 if end >= start else end + 24)
                for hour in hours:
                    hour = hour % 24
                    bus_coverage[DAYS_OF_WEEK[day_idx]][hour].append(bus)

            if (driver_schedule["driver"] < NUM_8HR_DRIVERS and 
                DAYS_OF_WEEK[day_idx] in ["Суббота", "Воскресенье"]):
                penalty += PENALTIES['weekend_work']

            if driver_schedule["driver"] >= NUM_8HR_DRIVERS:
                if day_idx > 0:
                    prev_day = schedule[day_idx - 1]
                    for prev_driver in prev_day:
                        if (prev_driver["driver"] == driver_schedule["driver"] and 
                            not prev_driver["rest"]):
                            penalty += PENALTIES['rest_violation']

    for day, coverage in bus_coverage.items():
        for hour, buses in coverage.items():
            buses_working = len(set(buses))
            shortage = max(0, NUM_BUSES - buses_working)
            
            if (hour in PEAK_HOURS['morning'] or 
                hour in PEAK_HOURS['evening']):
                penalty += shortage * PENALTIES['peak_hours_shortage']
            else:
                if 6 <= hour <= 22:
                    penalty += shortage * PENALTIES['bus_shortage']

    return bonus - penalty

def generate_random_schedule():
    schedule = []
    for day_idx, day in enumerate(DAYS_OF_WEEK):
        daily_schedule = []
        for driver_id in range(NUM_DRIVERS):
            if is_rest_day(driver_id, day_idx):
                daily_schedule.append({"driver": driver_id, "bus": None, "start": None, "end": None, "rest": True})
            else:
                bus = random.randint(0, NUM_BUSES - 1)
                shift_type = "8hr" if driver_id < NUM_8HR_DRIVERS else "12hr"
                start_time = random.randint(6, 10) if shift_type == "8hr" else random.randint(0, 23)
                end_time = (start_time + SHIFT_DURATIONS[shift_type]) % 24
                daily_schedule.append({
                    "driver": driver_id,
                    "bus": bus,
                    "start": start_time,
                    "end": end_time,
                    "rest": False
                })
        schedule.append(daily_schedule)
    return schedule


def mutate(schedule):
    day_idx = random.randint(0, len(schedule) - 1)
    driver_idx = random.randint(0, NUM_DRIVERS - 1)
    
    if not schedule[day_idx][driver_idx]["rest"]:
        shift_type = "8hr" if driver_idx < NUM_8HR_DRIVERS else "12hr"
        schedule[day_idx][driver_idx]["bus"] = random.randint(0, NUM_BUSES - 1)
        schedule[day_idx][driver_idx]["start"] = random.randint(6, 10) if shift_type == "8hr" else random.randint(0, 23)
        schedule[day_idx][driver_idx]["end"] = (schedule[day_idx][driver_idx]["start"] + SHIFT_DURATIONS[shift_type]) % 24
    
    return schedule

def genetic_algorithm():
    population = [generate_random_schedule() for _ in range(POPULATION_SIZE)]
    best_solution = None
    best_fitness = float('-inf')
    generations_without_improvement = 0
    
    for generation in range(MAX_GENERATIONS):
        current_mutation_rate = adaptive_mutation_rate(generations_without_improvement)
        
        # Вычисление фитнеса для всей популяции
        fitness_values = [fitness(schedule) for schedule in population]
        
        # Сортировка по фитнесу
        population_with_fitness = list(zip(population, fitness_values))
        population_with_fitness.sort(key=lambda x: x[1], reverse=True)
        
        current_best, current_fitness = population_with_fitness[0]
        
        if current_fitness > best_fitness:
            best_solution = current_best
            best_fitness = current_fitness
            generations_without_improvement = 0
        else:
            generations_without_improvement += 1
        
        next_generation = [ind for ind, _ in population_with_fitness[:ELITE_SIZE]]
        
        while len(next_generation) < POPULATION_SIZE:
            if random.random() < CROSSOVER_RATE:
                parents = random.sample(population_with_fitness[:int(POPULATION_SIZE * 0.4)], 2)
                child = two_point_crossover(parents[0][0], parents[1][0])
            else:
                child = random.choice(population_with_fitness[:int(POPULATION_SIZE * 0.4)])[0]

            if random.random() < current_mutation_rate:
                child = mutate(child)
            
            next_generation.append(child)

        population = next_generation
        
        if generation % 50 == 0:
            print(f"Поколение {generation}: Лучший фитнес = {current_fitness}")
            print(f"Поколений без улучшения: {generations_without_improvement}")

        if generations_without_improvement >= MAX_GENERATIONS_WITHOUT_IMPROVING:
            print(f"Досрочная остановка: нет улучшений в течение {MAX_GENERATIONS_WITHOUT_IMPROVING} поколений")
            break

    return best_solution

def generate_schedule_dataframe(schedule):
    driver_labels = [f"Водитель {d + 1}" for d in range(NUM_DRIVERS)]
    shifts = ["8hr" if d < NUM_8HR_DRIVERS else "12hr" for d in range(NUM_DRIVERS)]

    schedule_df = pd.DataFrame(index=driver_labels, columns=["Смена"] + DAYS_OF_WEEK)
    schedule_df["Смена"] = shifts

    for day_idx, daily_schedule in enumerate(schedule):
        for driver_schedule in daily_schedule:
            driver = f"Водитель {driver_schedule['driver'] + 1}"
            if driver_schedule["rest"]:
                schedule_df.loc[driver, DAYS_OF_WEEK[day_idx]] = "Отдых"
            else:
                bus = driver_schedule["bus"]
                start = driver_schedule["start"]
                end = driver_schedule["end"]
                schedule_df.loc[driver, DAYS_OF_WEEK[day_idx]] = f"Автобус {bus + 1} ({start:02d}:00 - {end:02d}:00)"

    return schedule_df.sort_values(by="Смена", ascending=False)

best_schedule = genetic_algorithm()
schedule_df = generate_schedule_dataframe(best_schedule)
print("\nЛучшее найденное расписание:")
schedule_df

Поколение 0: Лучший фитнес = -8755
Поколений без улучшения: 0
Поколение 50: Лучший фитнес = -9485
Поколений без улучшения: 45
Поколение 100: Лучший фитнес = -9370
Поколений без улучшения: 95
Поколение 150: Лучший фитнес = -8890
Поколений без улучшения: 145
Поколение 200: Лучший фитнес = -9845
Поколений без улучшения: 195
Поколение 250: Лучший фитнес = -9490
Поколений без улучшения: 245
Поколение 300: Лучший фитнес = -9385
Поколений без улучшения: 295
Поколение 350: Лучший фитнес = -9435
Поколений без улучшения: 345
Поколение 400: Лучший фитнес = -9560
Поколений без улучшения: 395
Поколение 450: Лучший фитнес = -9430
Поколений без улучшения: 445
Поколение 500: Лучший фитнес = -9595
Поколений без улучшения: 495
Поколение 550: Лучший фитнес = -8790
Поколений без улучшения: 545
Поколение 600: Лучший фитнес = -9985
Поколений без улучшения: 595
Поколение 650: Лучший фитнес = -9265
Поколений без улучшения: 645
Поколение 700: Лучший фитнес = -9155
Поколений без улучшения: 695
Поколение 750: Лу

,Смена,Понедельник,Вторник,Среда,Четверг,Пятница,Суббота,Воскресенье
Водитель 1,8hr,Автобус 8 (06:00 - 14:00),Автобус 1 (07:00 - 15:00),Автобус 4 (06:00 - 14:00),Автобус 2 (08:00 - 16:00),Автобус 4 (08:00 - 16:00),Отдых,Отдых
Водитель 2,8hr,Автобус 2 (09:00 - 17:00),Автобус 2 (10:00 - 18:00),Автобус 1 (06:00 - 14:00),Автобус 3 (09:00 - 17:00),Автобус 8 (06:00 - 14:00),Отдых,Отдых
Водитель 3,8hr,Автобус 2 (06:00 - 14:00),Автобус 7 (08:00 - 16:00),Автобус 2 (10:00 - 18:00),Автобус 6 (09:00 - 17:00),Автобус 5 (07:00 - 15:00),Отдых,Отдых
Водитель 4,8hr,Автобус 3 (07:00 - 15:00),Автобус 4 (08:00 - 16:00),Автобус 5 (09:00 - 17:00),Автобус 8 (06:00 - 14:00),Автобус 5 (09:00 - 17:00),Отдых,Отдых
Водитель 5,8hr,Автобус 2 (06:00 - 14:00),Автобус 2 (10:00 - 18:00),Автобус 2 (09:00 - 17:00),Автобус 3 (06:00 - 14:00),Автобус 2 (09:00 - 17:00),Отдых,Отдых
Водитель 6,8hr,Автобус 5 (07:00 - 15:00),Автобус 6 (08:00 - 16:00),Автобус 5 (09:00 - 17:00),Автобус 4 (10:00 - 18:00),Автобус 8 (10:00 - 18:00),Отдых,Отдых
Водитель 7,12hr,Автобус 2 (08:00 - 20:00),Отдых,Отдых,Автобус 1 (11:00 - 23:00),Отдых,Отдых,Автобус 6 (12:00 - 00:00)
Водитель 8,12hr,Автобус 2 (11:00 - 23:00),Отдых,Отдых,Автобус 4 (22:00 - 10:00),Отдых,Отдых,Автобус 4 (05:00 - 17:00)
Водитель 9,12hr,Отдых,Автобус 8 (10:00 - 22:00),Отдых,Отдых,Автобус 4 (03:00 - 15:00),Отдых,Отдых
Водитель 10,12hr,Отдых,Автобус 2 (08:00 - 20:00),Отдых,Отдых,Автобус 6 (09:00 - 21:00),Отдых,Отдых
